In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
import pandas as pd

In [2]:
store_number_lst = ['21726749', '34582591','1670762252']

In [3]:
store_number = store_number_lst[1] 
options = webdriver.ChromeOptions()
# options.add_argument('headless')#창 안 뜨게 설정
review_driver = webdriver.Chrome('./chromedriver.exe',options=options)# 크롬 경로
review_driver.implicitly_wait(3)

#url 접속
review_url = 'https://pcmap.place.naver.com/restaurant/'+store_number+'/home?from=map&amp;fromPanelNum=2&amp'
review_driver.get(review_url)
time.sleep(5)

In [4]:
html_cleaner = re.compile('<.*?>')

In [20]:
def store_info(soup):
    store_name = [] #이름
    store_location = [] #위치
    store_subway= [] #역으로부터의 거리
    store_hour=[] #영업시간
    store_detail1 = [""] #정보1
    store_detail2 = [] #정보2
    
    store_name=soup.find('span', {'class': '_3XamX'}) #이름
    store_name=re.sub(html_cleaner, "", str(store_name))
 
    store_location = soup.find('span',{'class':'_2yqUQ'}) #위치
    store_location = re.sub(html_cleaner,"",str(store_location))
        
    store_subway = soup.find('div',{'class':'_2P6sT'}) #역으로부터 거리
    store_subway = re.sub(html_cleaner,"",str(store_subway))
        
    open_time = soup.find_all('span',{'class':'_20pEw'}) #영업시간
    hour_string = " "
    for hour in open_time:
        hour = re.sub(html_cleaner,"",str(hour))
        hour_string += hour
    store_hour.append(hour_string)

    div_format = re.compile('<div class="_1h3B_">'+'[가-힣]+') 
    
    store_extrainfo = main_soup.find_all('div',{'class':'_1h3B_'}) #정보1
    for information in store_extrainfo:
        if div_format.search(str(information)):
            extra_info = re.sub(html_cleaner , "", str(information))
    store_detail1[0] = extra_info
           
     
    store_detail2 = soup.find('span',{'class':'WoYOw'}) #정보2
    store_detail2 = re.sub(html_cleaner,"",str(store_detail2))

                               
    #데이터 프레임으로 만들기 
    store_info = pd.DataFrame({'이름':store_name,'위치':store_location,'역으로부터 위치':store_subway,'영업시간':store_hour,'영업상세정보': store_detail1, '영업정보': store_detail2})
    
    return store_info 

In [7]:
main_html = review_driver.page_source
main_soup = BeautifulSoup(main_html,'html.parser')

In [21]:
store_info(main_soup)

,이름,위치,역으로부터 위치,영업시간,영업상세정보,영업정보
0,투썸플레이스 종각역점,서울 종로구 종로 72,1종각역 4번 출구에서127m,평일 08:00 - 23:30 토요일 08:30 - 23:30 일요일 09:00 ...,"단체석, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분, 국민지원금","종각역 10번 출구에서 10미터 거리에 있으며, YMCA본사 맞은편, 젊음의 거리 ..."
